In [1]:
from tuneapi import types as tt
from tuneapi import utils as tu
from tuneapi import apis as ta

In [2]:
# define a simple tool
find_movies_tool = tt.Tool(
    "find_movies",
    "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
    parameters=[
        tt.Tool.Prop(
            "location",
            "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616",
            "string",
        ),
        tt.Tool.Prop(
            "description",
            "Any kind of description including category or genre, title words, attributes, etc.",
            "string",
            required=True,
        ),
    ],
)
find_movies_tool

<Tool: find_movies>

In [3]:
# load existing from dictionaries
find_theatre = tt.Tool.from_dict({
    "name": "find_theaters",
    "description": "find theaters based on location and optionally movie title which is currently playing in theaters",
    "parameters": {
    "type": "object",
    "properties": {
        "location": {
        "type": "string",
        "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
        },
        "movie": {
        "type": "string",
        "description": "Any movie title"
        }
    },
    "required": [
        "location"
    ]
    }
})
find_theatre

<Tool: find_theaters>

In [4]:
# create a thread, keep adding messages like a list
thread = tt.Thread(
    tt.system("You are Tune Blob, a super star AI movie buff!"),
    tt.human("What movies are showing in Chennai tonight?"),
    tools = [
        find_movies_tool,
        find_theatre,
        tt.Tool(
            name = "get_showtimes",
            description = "Find the start times for movies playing in a specific theater",
            parameters = [
                tt.Tool.Prop(
                    name = "location",
                    description = "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616",
                    type = "string",
                    required = True
                ),
                tt.Tool.Prop(
                    name = "movie",
                    description = "Any movie title",
                    type = "string",
                    required = True
                ),
                tt.Tool.Prop(
                    name = "theater",
                    description = "Name of the theater",
                    type = "string",
                    required = True
                ),
                tt.Tool.Prop(
                    name = "date",
                    description = "Date for requested showtime",
                    type = "string",
                    required = True
                )
            ]
        )
    ]
)
thread

<Thread 'thread_49649210139099136' 
  <system: "You are Tune Blob, a super star AI movie buff!">
  <human: "What movies are showing in Chennai tonight?">
  <tools: ['find_movies', 'find_theaters', 'get_showtimes']>
>

In [6]:
# connect to a model from Tune Studio

# model = ta.TuneModel(
#     "yash/yash-gpt-4o",
#     org_id = "e41f60a3-06c9-4133-bc6d-127bc6f3f215"
# )

# or connect to few that have first class support
model = ta.Anthropic()
out = model.chat(thread)
print(out)

{'name': 'find_movies', 'arguments': {'description': 'movies playing in Chennai', 'location': 'Chennai'}}


In [7]:
# continue the thread
thread.append(
    tt.function_call(out)
)
thread

<Thread 'thread_49649210139099136' 
  <system: "You are Tune Blob, a super star AI movie buff!">
  <human: "What movies are showing in Chennai tonight?">
  <function_call: {"name": "find_movies", "arguments": {"description>
  <tools: ['find_movies', 'find_theaters', 'get_showtimes']>
>

In [8]:
# now attach a simple output after running the `find_movies` function
thread.append(tt.function_resp({
    "movie": "Barbie",
    "theaters": [
        {
            "name": "PVR - ECR",
            "address": "PVR: Heritage RSL ECR, Chennai - 600119"
        }, {
            "name": "Jazz Cinemas LUXE",
            "address": "No. 142, 2nd Floor, Phoenix Market City, Velachery Main Rd, Indira Gandhi Nagar, Velachery, Chennai, Tamil Nadu 600042"
        }
    ]
}))
thread

<Thread 'thread_49649210139099136' 
  <system: "You are Tune Blob, a super star AI movie buff!">
  <human: "What movies are showing in Chennai tonight?">
  <function_call: {"name": "find_movies", "arguments": {"description>
  <function_resp: {"movie": "Barbie", "theaters": [{"name": "PVR - E>
  <tools: ['find_movies', 'find_theaters', 'get_showtimes']>
>

In [11]:
# continue the conversation to get response from the model
out = model.chat(thread)
print(out)



According to the results, the movie "Barbie" is currently playing in theaters in Chennai, specifically at PVR - ECR and Jazz Cinemas LUXE.
